## Project prep
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    - cohort pulled from the sql table 
    - sort all the endpoints 
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
    - divided web dev and data science (4 different programs)
    - sort all the endpoints 
    - counts of cohorts accessing same endpoints
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
    - active status indicator 
    - filter by the student being active 
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
    - figure out what constitutes a suspicious IP address
        - any ip addresses from china?
        - figure out general area of ip address
    - how to recognize web scraping
    - SKIP 
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
    - how to classify the endpoints as web dev and not this might be crazy
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - active status indicator 
    - each row of a ping would have student_active column - Parker 
    - 
7. Which lessons are least accessed?
    - kind of related to 2
8. Anything else I should be aware of?

1 is related to 7 and 2 

3 is relate to 6

anything extra we find

### Data Frame Must Haves
- log data
- cohort data
- Join on cohort id (Left Join) - Lupe has the join and the drop columns 
- column that has the active student at time of request (yes or no)
- need key for all the program ids (is_data_science maybe) 
- endpoint as a web dev, or data science 
    - Anything came before ada's start date is webdev


In [2]:
# prepare file for anomaly detection exercises
import pandas as pd
import numpy as np

# Vis tools
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans 

# defining some functions to make it easier. will go in Wrangle function
from env import host, password, user
import os


In [3]:
###################### Getting database Url ################
def get_db_url(db_name, user=user, host=host, password=password):
    """
        This helper function takes as default the user host and password from the env file.
        You must input the database name. It returns the appropriate URL to use in connecting to a database.
    """
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
    return url

######################### get generic data #########################
def get_any_data(database, sql_query):
    '''
    put in the query and the database and get the data you need in a dataframe
    '''

    return pd.read_sql(sql_query, get_db_url(database))

In [5]:
def acquire_curriculum_logs(csv_name = "anonymized-curriculum-access-07-2021.txt"):
    '''
    This function reads the csv of curriculum access logs
    If the file name needs to be changed change the default arguement
    '''
    # assign column names to use
    colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
    # read csv
    df = pd.read_csv(csv_name, 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
    return df

def make_datetime_index(df, col_name):
    '''
    This function takes in a dataframe 
    A column name of the column that is your date (as string)
    Performs basic to_datetime conversion and sets tha column as the index
    '''
    
    df[col_name] = pd.to_datetime(df[col_name])

    df = df.set_index(col_name)
    
    return df

def prep(df):
    '''
    This function takes in the curriculum log data, and the user number you want to look at
    returns a dataframe of the pages they accessed and when
    '''
    #df = df[df.user_id == user]
    df = make_datetime_index(df, 'date')
    pages = df['endpoint'].resample('d').count()
    return pages

In [6]:
df = acquire_curriculum_logs() 

<ipython-input-5-5789d8fa33d1>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(csv_name,


In [8]:
dbase = 'curriculum_logs'
sqlquery = 'SELECT * FROM cohorts'

cohorts = get_any_data(dbase, sqlquery)